<a href="https://colab.research.google.com/github/parus-cristatus/tolokapizza/blob/main/tolokaapi/get_attachments_and_upload_to_cloud.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install boto3

In [9]:
import boto3
import time
import requests
import threading

In [4]:
API_KEY = 'YYYYYYYYYYYYYYYYYYYYY'
SECRET_ACCESS_KEY = 'ZZZZZZZZZZZZZZZZZZZZZZZZZZZZZ'
BUCKET_NAME = 'my-bucket'

TOLOKA_TOKEN = 'XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX'

In [10]:
session = boto3.session.Session()

s3 = session.client(
    service_name='s3',
    endpoint_url='https://storage.yandexcloud.net',
    aws_access_key_id = API_KEY,
    aws_secret_access_key = SECRET_ACCESS_KEY
)

In [11]:
def check_runtime(func):
    def wrapped(args):
        start = time.time()
        func(args)
        end = time.time()
        print()
        print(f'Function takes: {round(end - start, 3)} seconds')
    
    return wrapped

In [12]:
def change_name(id: str, file_name: str) -> str:
    name = file_name.split('.')[0]
    postfix = id.split('-')[4]
    format = file_name.split('.')[1]

    return f"{name}_{postfix}.{format}"


def get_call(token: str, call: str, params: dict = {}) -> dict:
    headers = {
        "Authorization": "OAuth " + token
    }

    response = requests.get(url=f'https://sandbox.toloka.yandex.com/api/v1/{call}', headers=headers, params=params)
    response.raise_for_status()

    return response


def get_attachments(pool_id: str) -> dict:
    param = {
        "pool_id": pool_id
        }
    response = get_call(token=TOLOKA_TOKEN, call=f'attachments', params=param)

    return response.json()


def get_attachment_ids(data: dict) -> dict:
    return {i['id']: i['name'] for i in data['items']}


def upload_to_cloud(bucket: str, file_name: str, content: dict):
        print(f'Direct link: https://storage.yandexcloud.net/{bucket}/{file_name}')
        s3.put_object(Bucket=bucket, Key=file_name, Body=content.content, StorageClass='STANDARD')
        


def upload_to_anonfiles(file_name: str, content: dict):
    """Upload file to https://anonfiles.com/"""

    info = requests.post('https://api.anonfiles.com/upload', files={'file': (f'{file_name}', content.content)})
    print(f"Direct link: {info.json()['data']['file']['url']['full']}")


@check_runtime
def upload_attachments(attachments: dict):
    for id, name in attachments.items():
        response = get_call(token=TOLOKA_TOKEN, call=f'attachments/{id}/download')
        file_name = change_name(id=id, file_name=name)

        threading.Thread(target=upload_to_cloud, args=(BUCKET_NAME, file_name, response)).start()

        # s3.put_object(Bucket=BUCKET_NAME, Key=file_name, Body=response.content, StorageClass='STANDARD')
        # print(f'Direct link: https://storage.yandexcloud.net/{BUCKET_NAME}/{file_name}')

        # info = requests.post('https://api.anonfiles.com/upload', files={'file': (f'{file_name}', response.content)})
        # print(f"Direct link: {info.json()['data']['file']['url']['full']}")

        #threading.Thread(target=upload_to_anonfiles, args=(file_name, response)).start()

In [ ]:
pool_id = '1193556'
json_data = get_attachments(pool_id=pool_id)
attachments = get_attachment_ids(json_data)
upload_attachments(attachments)